In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import joblib, pickle
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report


In [5]:
'''
Objective:
1.
Develop a Logistic Regression model using customer data to predict ad clicks.
2.
Enhance marketing strategies through data-driven insights.
3.
Improve ad targeting by accurately predicting user behavior.
Constraints:
1.
Ensure compliance with privacy laws and data protection regulations.
2.
Achieve high accuracy in ad click predictions to optimize marketing efforts.
3.
Maintain interpretability while handling large volumes of customer data efficiently.
'''


add = pd.read_csv('advertising.csv')
add

Daily_Time_ Spent _on_Site  Age  Area_Income  Daily Internet Usage  \
0                         68.95   35     61833.90                256.09   
1                         80.23   31     68441.85                193.77   
2                         69.47   26     59785.94                236.50   
3                         74.15   29     54806.18                245.89   
4                         68.37   35     73889.99                225.58   
..                          ...  ...          ...                   ...   
995                       72.97   30     71384.57                208.58   
996                       51.30   45     67782.17                134.42   
997                       51.63   51     42415.72                120.37   
998                       55.55   19     41920.79                187.95   
999                       45.01   26     29875.80                178.35   

                             Ad_Topic_Line            City  Male  \
0       Cloned 5thgeneration orchestration     Wrightburgh     0   
1       Monitored national standardization       West Jodi     1   
2         Organic bottom-line service-desk        Davidton     0   
3    Triple-buffered reciprocal time-frame  West Terrifurt     1   
4            Robust logistical utilization    South Manuel     0   
..                                     ...             ...   ...   
995          Fundamental modular algorithm       Duffystad     1   
996        Grass-roots cohesive monitoring     New Darlene     1   
997           Expanded intangible solution   South Jessica     1   
998   Proactive bandwidth-monitored policy     West Steven     0   
999        Virtual 5thgeneration emulation     Ronniemouth     0   

                    Country         Timestamp  Clicked_on_Ad  
0                   Tunisia  27-03-2016 00:53              0  
1                     Nauru  04-04-2016 01:39              0  
2                San Marino  13-03-2016 20:35              0  
3                     Italy  10-01-2016 02:31              0  
4                   Iceland  03-06-2016 03:36              0  
..                      ...               ...            ...  
995                 Lebanon  11-02-2016 21:49              1  
996  Bosnia and Herzegovina  22-04-2016 02:07              1  
997                Mongolia  01-02-2016 17:24              1  
998               Guatemala  24-03-2016 02:35              0  
999                  Brazil  03-06-2016 21:43              1  

[1000 rows x 10 columns]

In [3]:
add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Daily_Time_ Spent _on_Site  1000 non-null   float64
 1   Age                         1000 non-null   int64  
 2   Area_Income                 1000 non-null   float64
 3   Daily Internet Usage        1000 non-null   float64
 4   Ad_Topic_Line               1000 non-null   object 
 5   City                        1000 non-null   object 
 6   Male                        1000 non-null   int64  
 7   Country                     1000 non-null   object 
 8   Timestamp                   1000 non-null   object 
 9   Clicked_on_Ad               1000 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 78.3+ KB


In [4]:
add.isna().sum()

Daily_Time_ Spent _on_Site    0
Age                           0
Area_Income                   0
Daily Internet Usage          0
Ad_Topic_Line                 0
City                          0
Male                          0
Country                       0
Timestamp                     0
Clicked_on_Ad                 0
dtype: int64

In [11]:
x = add.drop(['Ad_Topic_Line', 'City', 'Timestamp','Country','Clicked_on_Ad'], axis = 1)
y = add['Clicked_on_Ad']

In [12]:
#imputation
num_pipeline = Pipeline(steps =[('impute', SimpleImputer(strategy ='mean'))])
num_preprocessor = ColumnTransformer(transformers=[('numeric', num_pipeline, x.columns)])
impute_data = num_preprocessor.fit(x)
impute_df = pd.DataFrame(impute_data.transform(x), columns = x.columns)

In [14]:
winsor = Winsorizer(capping_method ='iqr', tail = 'both', fold = 1.5, variables = list(impute_df.columns))
outlier_pipeline = Pipeline(steps =[('wins', winsor)])
outlier_data = outlier_pipeline.fit(impute_df)
outlier_df = pd.DataFrame(outlier_data.transform(impute_df), columns = x.columns)

In [16]:
scale_pipeline = Pipeline(steps =[('scale', MinMaxScaler())])
scale_preprocessor = ColumnTransformer(transformers=[('scale', scale_pipeline, impute_df.columns)])
scale_data = scale_preprocessor.fit(outlier_df)
scale_df = pd.DataFrame(scale_data.transform(outlier_df), columns = x.columns)

In [17]:
#model building

In [18]:
x_train, x_test, y_train, y_test = train_test_split(scale_df, y, test_size = 0.2, random_state = 0)

In [19]:
joblib.dump(impute_data, 'lR_im_assign')
joblib.dump(outlier_data, 'lr_outlier_assign')
joblib.dump(scale_data, 'lr_scale_assign')

['lr_scale_assign']

In [20]:
### Statsmodel 
# Building the model and fitting the data


In [21]:
logit = sm.Logit(y, scale_df).fit()

Optimization terminated successfully.
         Current function value: 0.247186
         Iterations 8


In [22]:
logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Clicked_on_Ad   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      995
Method:                           MLE   Df Model:                            4
Date:                Tue, 27 Feb 2024   Pseudo R-squ.:                  0.6434
Time:                        14:57:46   Log-Likelihood:                -247.19
converged:                       True   LL-Null:                       -693.15
Covariance Type:            nonrobust   LLR p-value:                9.367e-192
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Daily_Time_ Spent _on_Site    -3.6287      0.469     -7.736      0.000      -4.548      -2.709
Age                            9.6350      0.690     13.974      0.000       8.284      10.986
Area_Income                    0.3653      0.447      0.817      0.414      -0.511       1.242
Daily Internet Usage          -5.3610      0.541     -9.905      0.000      -6.422      -4.300
Male                           0.6075      0.235      2.583      0.010       0.147       1.068
==============================================================================================
"""

In [23]:
logit.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Results: Logit
==========================================================================
Model:                  Logit              Method:             MLE        
Dependent Variable:     Clicked_on_Ad      Pseudo R-squared:   0.643      
Date:                   2024-02-27 14:58   AIC:                504.3711   
No. Observations:       1000               BIC:                528.9099   
Df Model:               4                  Log-Likelihood:     -247.19    
Df Residuals:           995                LL-Null:            -693.15    
Converged:              1.0000             LLR p-value:        9.3668e-192
No. Iterations:         8.0000             Scale:              1.0000     
--------------------------------------------------------------------------
                            Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
--------------------------------------------------------------------------
Daily_Time_ Spent _on_Site -3.6287   0.4691 -7.7360 0.0000 -4.5481 -2.7094
Age                         9.6350   0.6895 13.9736 0.0000  8.2836 10.9864
Area_Income                 0.3653   0.4472  0.8168 0.4140 -0.5112  1.2419
Daily Internet Usage       -5.3610   0.5413 -9.9048 0.0000 -6.4218 -4.3002
Male                        0.6075   0.2352  2.5829 0.0098  0.1465  1.0684
==========================================================================

"""

In [24]:
pred = logit.predict(scale_df)

In [25]:
pred

0      0.038256
1      0.102698
2      0.009033
3      0.017536
4      0.106671
         ...   
995    0.082264
996    0.991429
997    0.998355
998    0.018381
999    0.184889
Length: 1000, dtype: float64

In [30]:
# ROC Curve to identify the appropriate cutoff value

fpr, tpr, thresholds= roc_curve(y, pred)

In [31]:
optimal_idx = np.argmax(tpr-fpr)

In [33]:
optimal_threshold = thresholds[optimal_idx]
optimal_threshold

0.4307151071242358

In [35]:
auc = metrics.auc(fpr, tpr)


In [36]:
auc

0.9614

In [40]:
# Classification report

import sklearn.metrics as skmet
skmet.classification_report(pred, y)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [41]:
#model evaluation

In [42]:
model = sm.Logit(y_train, x_train).fit()

Optimization terminated successfully.
         Current function value: 0.254634
         Iterations 8


In [43]:
#prediction

In [50]:
# Validate on Test data

pred_test = model.predict(x_test)

In [51]:
pred_test

993    0.049491
859    0.191654
298    0.030896
553    0.958218
672    0.022554
         ...   
679    0.947739
722    0.985461
215    0.065251
653    0.188645
150    0.212934
Length: 200, dtype: float64

ValueError: unknown format is not supported